## This notebook loads a dataframe with the 2467 records in the Notices of Inventory table on the National Park Service website and adds columns for JSON and full text URLs. The final result is saved as 'inv_urls.csv'

Note that this notebook was originally named "Saving Notices of Inventory Completion.ipynb."

In [46]:
import pandas as pd
import numpy as np
import pathlib
import requests
import json

In [2]:
'''
Loading Notices of Inventory via Federal Register

fedreg_notices_of_inventory.csv downloaded from Federal Register 2/13/20
search for "Inventory Completion"
agency: National Park Service
type: Notice
'''

data_path = pathlib.Path.cwd() / 'datasets' / 'fedreg_notices_of_inventory.csv'
fed_inventories = pd.read_csv(data_path)
fed_inventories['html_url']

0      https://www.federalregister.gov/documents/2020...
1      https://www.federalregister.gov/documents/2019...
2      https://www.federalregister.gov/documents/2019...
3      https://www.federalregister.gov/documents/2019...
4      https://www.federalregister.gov/documents/2019...
                             ...                        
995    https://www.federalregister.gov/documents/2011...
996    https://www.federalregister.gov/documents/2011...
997    https://www.federalregister.gov/documents/2013...
998    https://www.federalregister.gov/documents/2013...
999    https://www.federalregister.gov/documents/2013...
Name: html_url, Length: 1000, dtype: object

#### Problem: Only 1000 results included in Federal Register's dataset, although 2697 results found in search. 

#### Instead we'll try to start building our dataset from the NPS site. (We can try to load in the documents that exist in the Federal Register but not in NPS' table (i.e. corrections) by JSON linkage later.)

In [3]:
'''
Loading notices of inventory via national park service (nps) json

Found URL for JSON with data via pagesource on 
https://www.nps.gov/subjects/nagpra/notices-of-inventory-completion.htm

Accessed 2/13/20.
'''

url = 'https://www.nps.gov/common/uploads/sortable_dataset/nagpra/F8663396-E1B9-7C54-8C15C08D2D0702C4/F8663396-E1B9-7C54-8C15C08D2D0702C4.json'
response=requests.get(url)
inventories_dict = json.loads(response.content)
nps_inventories = pd.DataFrame(data= \
                               json.loads(response.content)['DATA'], \
                               columns=json.loads(response.content)['COLUMNS'])
nps_inventories

,Publication Date,Title,Link
0,12/9/2019,"Sam Noble Oklahoma Museum of Natural History, ...",https://www.federalregister.gov/documents/2019...
1,11/27/2019,"Tennessee Valley Authority, Knoxville, TN",https://www.federalregister.gov/documents/2019...
2,11/27/2019,"Tennessee Valley Authority, Knoxville, TN",https://www.federalregister.gov/documents/2019...
3,11/27/2019,"University of California, Santa Cruz, Santa Cr...",https://www.federalregister.gov/documents/2019...
4,11/27/2019,"Los Angeles Pierce College, Woodland Hills, CA",https://www.federalregister.gov/documents/2019...
...,...,...,...
2462,7/2/1994,Notice of Inventory Completion for Native Amer...,https://www.federalregister.gov/documents/1994...
2463,2/28/1994,Inventory Completion of Native American Human ...,https://www.federalregister.gov/documents/1994...
2464,2/28/1994,Inventory Completion for Native American Human...,https://www.federalregister.gov/documents/1994...
2465,2/25/1994,Notice of Completion of Inventory of Native Am...,https://www.federalregister.gov/documents/1994...


#### Dataset from National Park Services returns 2467 results

In [92]:
'''
Trying to open Federal Register document
via NPS Notices of Inventories dataframe

Goal is to get metadata as well as full text for each record

Testing on first record
'''

test_url = nps_inventories.loc[0]['Link']
test_response=requests.get(test_url)
test_response.content

b'<!DOCTYPE html>\n  <head>\n    <title>\n      Federal Register\n       :: \n      Notice of Inventory Completion: Sam Noble Oklahoma Museum of Natural History, Norman, OK\n    </title>\n\n    <!-- Tell search engines to use the proper hostname and protocol -->\n    <link rel="canonical" href="https://www.federalregister.gov/documents/2019/12/09/2019-26434/notice-of-inventory-completion-sam-noble-oklahoma-museum-of-natural-history-norman-ok" />\n\n    <link rel="Shortcut Icon" href="/favicon.ico">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n    <script src="https://use.typekit.net/rgf3yox.js"></script>\n<script type="text/javascript">\n  try {\n    Typekit.load({\n      async: true,\n      active: function() {\n        $(\'body\').trigger(\'typekit-active\');\n      }\n    })\n  } catch(e) {}\n</script>\n\n\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta name="keywords" content="government federal register executive 

#### Appending '.json' to URL does not direct to json as expected. 

#### HTML (saved as test_response.content) contains URLs for JSON (containing metadata) and XML (containing original full text). 

#### The question now is how to isolate these URLs of interest from the HTML.  

In [94]:
'''
Using magic command to load python script
(Doing it this way because I had trouble with importing & passing arguments)

Need to talk to DE re separating the script.
Renamed script from Download_JSON.py because that was no longer descriptive
'''

# %load ../urlgetter.py
from bs4 import BeautifulSoup
#import sys
#html_page_source=sys.argv[1]

def get_urls (html):
    '''
    Parses HTML to return URLs of interest
    (i.e., links to pages with JSON and with full text)
    '''
    soup = BeautifulSoup(html, "html.parser")
    fulltext_url = None
    json_url = None
    for link in soup.find_all('a'):
        if link.string == None:
            continue
        if fulltext_url != None and json_url != None:
            break
        if link.string == 'JSON: Normalized attributes and metadata':
            json_url = link.get('href')
            
        '''
        could refine logic here. "text" covers 
        "XML: Original full text XML" 
        as well as "basic text format"
        '''
        if 'text' in link.string.lower(): 
            fulltext_url = link.get('href')
    return json_url, fulltext_url


In [95]:
'''
Adding and filling 'json_url' and 'fulltext_url' columns to nps_inventories

Takes a long time to run. There probably exists a more efficient way than .apply
Only need to run it once, though.
'''

def fill_urls(row):
    fedreg_url = row['Link']
    test_response=requests.get(fedreg_url)
    html_page_source = str(test_response.content)[2:-1]
    #%run -i Download_JSON
    json_url, fulltext_url = get_urls(html_page_source)
    row['json_url'] = json_url
    row['fulltext_url'] = fulltext_url
    print(row.name, json_url, fulltext_url)
    return row

nps_inventories['json_url'] = np.nan
nps_inventories['fulltext_url'] = np.nan


nps_inventories = nps_inventories.apply(fill_urls, axis=1)

0 https://www.federalregister.gov/api/v1/documents/2019-26434?publication_date=2019-12-09 https://www.federalregister.gov/documents/full_text/xml/2019/12/09/2019-26434.xml
0 https://www.federalregister.gov/api/v1/documents/2019-26434?publication_date=2019-12-09 https://www.federalregister.gov/documents/full_text/xml/2019/12/09/2019-26434.xml
1 https://www.federalregister.gov/api/v1/documents/2019-25731?publication_date=2019-11-27 https://www.federalregister.gov/documents/full_text/xml/2019/11/27/2019-25731.xml
2 https://www.federalregister.gov/api/v1/documents/2019-25730?publication_date=2019-11-27 https://www.federalregister.gov/documents/full_text/xml/2019/11/27/2019-25730.xml
3 https://www.federalregister.gov/api/v1/documents/2019-25734?publication_date=2019-11-27 https://www.federalregister.gov/documents/full_text/xml/2019/11/27/2019-25734.xml
4 https://www.federalregister.gov/api/v1/documents/2019-25729?publication_date=2019-11-27 https://www.federalregister.gov/documents/full_tex

47 https://www.federalregister.gov/api/v1/documents/2019-16680?publication_date=2019-08-05 https://www.federalregister.gov/documents/full_text/xml/2019/08/05/2019-16680.xml
48 https://www.federalregister.gov/api/v1/documents/2019-16681?publication_date=2019-08-05 https://www.federalregister.gov/documents/full_text/xml/2019/08/05/2019-16681.xml
49 https://www.federalregister.gov/api/v1/documents/2019-16685?publication_date=2019-08-05 https://www.federalregister.gov/documents/full_text/xml/2019/08/05/2019-16685.xml
50 https://www.federalregister.gov/api/v1/documents/2019-15439?publication_date=2019-07-19 https://www.federalregister.gov/documents/full_text/xml/2019/07/19/2019-15439.xml
51 https://www.federalregister.gov/api/v1/documents/2019-15437?publication_date=2019-07-19 https://www.federalregister.gov/documents/full_text/xml/2019/07/19/2019-15437.xml
52 https://www.federalregister.gov/api/v1/documents/2019-15441?publication_date=2019-07-19 https://www.federalregister.gov/documents/fu

95 https://www.federalregister.gov/api/v1/documents/2019-08741?publication_date=2019-04-30 https://www.federalregister.gov/documents/full_text/xml/2019/04/30/2019-08741.xml
96 https://www.federalregister.gov/api/v1/documents/2019-08742?publication_date=2019-04-30 https://www.federalregister.gov/documents/full_text/xml/2019/04/30/2019-08742.xml
97 https://www.federalregister.gov/api/v1/documents/2019-08743?publication_date=2019-04-30 https://www.federalregister.gov/documents/full_text/xml/2019/04/30/2019-08743.xml
98 https://www.federalregister.gov/api/v1/documents/2019-08740?publication_date=2019-04-30 https://www.federalregister.gov/documents/full_text/xml/2019/04/30/2019-08740.xml
99 https://www.federalregister.gov/api/v1/documents/2019-08589?publication_date=2019-04-29 https://www.federalregister.gov/documents/full_text/xml/2019/04/29/2019-08589.xml
100 https://www.federalregister.gov/api/v1/documents/2019-08594?publication_date=2019-04-29 https://www.federalregister.gov/documents/f

143 https://www.federalregister.gov/api/v1/documents/2019-03578?publication_date=2019-03-01 https://www.federalregister.gov/documents/full_text/xml/2019/03/01/2019-03578.xml
144 https://www.federalregister.gov/api/v1/documents/2019-03568?publication_date=2019-03-01 https://www.federalregister.gov/documents/full_text/xml/2019/03/01/2019-03568.xml
145 https://www.federalregister.gov/api/v1/documents/2019-03582?publication_date=2019-03-01 https://www.federalregister.gov/documents/full_text/xml/2019/03/01/2019-03582.xml
146 https://www.federalregister.gov/api/v1/documents/2019-03581?publication_date=2019-03-01 https://www.federalregister.gov/documents/full_text/xml/2019/03/01/2019-03581.xml
147 https://www.federalregister.gov/api/v1/documents/2019-03588?publication_date=2019-03-01 https://www.federalregister.gov/documents/full_text/xml/2019/03/01/2019-03588.xml
148 https://www.federalregister.gov/api/v1/documents/2019-03575?publication_date=2019-03-01 https://www.federalregister.gov/docume

191 https://www.federalregister.gov/api/v1/documents/2018-25123?publication_date=2018-11-19 https://www.federalregister.gov/documents/full_text/xml/2018/11/19/2018-25123.xml
192 https://www.federalregister.gov/api/v1/documents/2018-25125?publication_date=2018-11-19 https://www.federalregister.gov/documents/full_text/xml/2018/11/19/2018-25125.xml
193 https://www.federalregister.gov/api/v1/documents/2018-25124?publication_date=2018-11-19 https://www.federalregister.gov/documents/full_text/xml/2018/11/19/2018-25124.xml
194 https://www.federalregister.gov/api/v1/documents/2018-24764?publication_date=2018-11-14 https://www.federalregister.gov/documents/full_text/xml/2018/11/14/2018-24764.xml
195 https://www.federalregister.gov/api/v1/documents/2018-24765?publication_date=2018-11-14 https://www.federalregister.gov/documents/full_text/xml/2018/11/14/2018-24765.xml
196 https://www.federalregister.gov/api/v1/documents/2018-24767?publication_date=2018-11-14 https://www.federalregister.gov/docume

...

2216 https://www.federalregister.gov/api/v1/documents/00-18464?publication_date=2000-07-21 https://www.federalregister.gov/documents/full_text/xml/2000/07/21/00-18464.xml
2217 https://www.federalregister.gov/api/v1/documents/00-18138?publication_date=2000-07-18 https://www.federalregister.gov/documents/full_text/xml/2000/07/18/00-18138.xml
2218 https://www.federalregister.gov/api/v1/documents/00-18136?publication_date=2000-07-18 https://www.federalregister.gov/documents/full_text/xml/2000/07/18/00-18136.xml
2219 https://www.federalregister.gov/api/v1/documents/00-18137?publication_date=2000-07-18 https://www.federalregister.gov/documents/full_text/xml/2000/07/18/00-18137.xml
2220 https://www.federalregister.gov/api/v1/documents/00-14488?publication_date=2000-06-08 https://www.federalregister.gov/documents/full_text/xml/2000/06/08/00-14488.xml
2221 https://www.federalregister.gov/api/v1/documents/00-14489?publication_date=2000-06-08 https://www.federalregister.gov/documents/full_text/xm

2267 https://www.federalregister.gov/api/v1/documents/99-22260?publication_date=1999-08-27 https://www.govinfo.gov/content/pkg/FR-1999-08-27/html/99-22260.htm
2268 https://www.federalregister.gov/api/v1/documents/99-22167?publication_date=1999-08-26 https://www.govinfo.gov/content/pkg/FR-1999-08-26/html/99-22167.htm
2269 https://www.federalregister.gov/api/v1/documents/99-22166?publication_date=1999-08-26 https://www.govinfo.gov/content/pkg/FR-1999-08-26/html/99-22166.htm
2270 https://www.federalregister.gov/api/v1/documents/99-22164?publication_date=1999-08-26 https://www.govinfo.gov/content/pkg/FR-1999-08-26/html/99-22164.htm
2271 https://www.federalregister.gov/api/v1/documents/99-22165?publication_date=1999-08-26 https://www.govinfo.gov/content/pkg/FR-1999-08-26/html/99-22165.htm
2272 https://www.federalregister.gov/api/v1/documents/99-22162?publication_date=1999-08-26 https://www.govinfo.gov/content/pkg/FR-1999-08-26/html/99-22162.htm
2273 https://www.federalregister.gov/api/v1/do

2319 https://www.federalregister.gov/api/v1/documents/98-31482?publication_date=1998-11-25 https://www.govinfo.gov/content/pkg/FR-1998-11-25/html/98-31482.htm
2320 https://www.federalregister.gov/api/v1/documents/98-31350?publication_date=1998-11-24 https://www.govinfo.gov/content/pkg/FR-1998-11-24/html/98-31350.htm
2321 https://www.federalregister.gov/api/v1/documents/98-30683?publication_date=1998-11-18 https://www.govinfo.gov/content/pkg/FR-1998-11-18/html/98-30683.htm
2322 https://www.federalregister.gov/api/v1/documents/98-30681?publication_date=1998-11-17 https://www.govinfo.gov/content/pkg/FR-1998-11-17/html/98-30681.htm
2323 https://www.federalregister.gov/api/v1/documents/98-30680?publication_date=1998-11-17 https://www.govinfo.gov/content/pkg/FR-1998-11-17/html/98-30680.htm
2324 https://www.federalregister.gov/api/v1/documents/98-29093?publication_date=1998-10-30 https://www.govinfo.gov/content/pkg/FR-1998-10-30/html/98-29093.htm
2325 https://www.federalregister.gov/api/v1/do

2371 https://www.federalregister.gov/api/v1/documents/98-7580?publication_date=1998-03-24 https://www.govinfo.gov/content/pkg/FR-1998-03-24/html/98-7580.htm
2372 https://www.federalregister.gov/api/v1/documents/98-6138?publication_date=1998-03-10 https://www.govinfo.gov/content/pkg/FR-1998-03-10/html/98-6138.htm
2373 https://www.federalregister.gov/api/v1/documents/98-5918?publication_date=1998-03-09 https://www.govinfo.gov/content/pkg/FR-1998-03-09/html/98-5918.htm
2374 https://www.federalregister.gov/api/v1/documents/98-5916?publication_date=1998-03-09 https://www.govinfo.gov/content/pkg/FR-1998-03-09/html/98-5916.htm
2375 https://www.federalregister.gov/api/v1/documents/98-5408?publication_date=1998-03-03 https://www.govinfo.gov/content/pkg/FR-1998-03-03/html/98-5408.htm
2376 https://www.federalregister.gov/api/v1/documents/98-4684?publication_date=1998-02-24 https://www.govinfo.gov/content/pkg/FR-1998-02-24/html/98-4684.htm
2377 https://www.federalregister.gov/api/v1/documents/98-4

2423 https://www.federalregister.gov/api/v1/documents/97-15226?publication_date=1997-06-13 https://www.govinfo.gov/content/pkg/FR-1997-06-13/html/97-15226.htm
2424 https://www.federalregister.gov/api/v1/documents/97-15225?publication_date=1997-06-11 https://www.govinfo.gov/content/pkg/FR-1997-06-11/html/97-15225.htm
2425 https://www.federalregister.gov/api/v1/documents/97-15228?publication_date=1997-06-11 https://www.govinfo.gov/content/pkg/FR-1997-06-11/html/97-15228.htm
2426 https://www.federalregister.gov/api/v1/documents/97-15224?publication_date=1997-06-11 https://www.govinfo.gov/content/pkg/FR-1997-06-11/html/97-15224.htm
2427 https://www.federalregister.gov/api/v1/documents/97-14310?publication_date=1997-06-02 https://www.govinfo.gov/content/pkg/FR-1997-06-02/html/97-14310.htm
2428 https://www.federalregister.gov/api/v1/documents/97-14309?publication_date=1997-06-02 https://www.govinfo.gov/content/pkg/FR-1997-06-02/html/97-14309.htm
2429 https://www.federalregister.gov/api/v1/do

In [98]:
print(len(nps_inventories.json_url.unique())==len(nps_inventories))
print(len(nps_inventories.fulltext_url.unique())==len(nps_inventories))

True
True


Each row has unique entries for the JSON & full text URLS.

In [105]:
#save as csv called inv_urls.csv
nps_inventories.to_csv('datasets/inv_urls.csv', index=False)

In [110]:
#from now on, we can read in the dataframe of URLs from the csv
inv_urls = pd.read_csv('datasets/inv_urls.csv')

In [111]:
inv_urls

,Publication Date,Title,Link,json_url,fulltext_url
0,12/9/2019,"Sam Noble Oklahoma Museum of Natural History, ...",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
1,11/27/2019,"Tennessee Valley Authority, Knoxville, TN",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
2,11/27/2019,"Tennessee Valley Authority, Knoxville, TN",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
3,11/27/2019,"University of California, Santa Cruz, Santa Cr...",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
4,11/27/2019,"Los Angeles Pierce College, Woodland Hills, CA",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
...,...,...,...,...,...
2462,7/2/1994,Notice of Inventory Completion for Native Amer...,https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-07...
2463,2/28/1994,Inventory Completion of Native American Human ...,https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-02...
2464,2/28/1994,Inventory Completion for Native American Human...,https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-02...
2465,2/25/1994,Notice of Completion of Inventory of Native Am...,https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-02...
